# SPICED Academy /// Project Week 08 /// Markov Simulation

***

## I. Define goal

***

## II. Import libraries and data

In [29]:
#data processing
import pandas as pd
import numpy as np

#data visualization
import seaborn as sns
import matplotlib.pyplot as plt

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [55]:
#import relevant data
mon = pd.read_csv('data/monday.csv', sep = ';', index_col = 'customer_no')
tue = pd.read_csv('data/tuesday.csv', sep = ';', index_col = 'customer_no')
wed = pd.read_csv('data/wednesday.csv', sep = ';', index_col = 'customer_no')
thur = pd.read_csv('data/thursday.csv', sep = ';', index_col = 'customer_no')
fri = pd.read_csv('data/friday.csv', sep = ';', index_col = 'customer_no')

mon.head()

,timestamp,location
customer_no,,
1,2019-09-02 07:03:00,dairy
2,2019-09-02 07:03:00,dairy
3,2019-09-02 07:04:00,dairy
4,2019-09-02 07:04:00,dairy
5,2019-09-02 07:04:00,spices


***

## III. Exploratory Data Analysis

*Exploring: Location*

In [31]:
#Unique location values
mon['location'].unique()

array(['dairy', 'spices', 'fruit', 'checkout', 'drinks'], dtype=object)

In [47]:
mon['location'].value_counts()

checkout    1437
fruit       1005
dairy        895
drinks       797
spices       750
Name: location, dtype: int64

In [49]:
mon.dtypes

timestamp      object
customer_no     int64
location       object
dtype: object

In [ ]:
mon['before']
mon['after']

***

## Transition probabilities 

*State diagram*

*Display the transition probability matrix*

*Visualize the probabilities as a heat map*

In [ ]:
#calculating probabilities 
pd.crosstab(df['before'], df['after'], normalize=0)

***

## Monte Carlo Markov Chain

***

## CSV Output